### LIBRARIES

In [1]:
# LIBRARIES
vim: syntax=pythonimport math
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import chime        # Notification sounds
from matplotlib import rc
from tensorflow import keras
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, ShuffleSplit, GroupShuffleSplit
from sklearn import preprocessing
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from functools import partial
%load_ext chime

### SETTING STYLE

In [2]:
# STYLE
# Plot/table options
desired_width = 400
pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns', 12)
rc('font', **{'family': 'serif', 'serif': ['Times New Roman']})
plt.rcParams['font.size'] = 14

# Colors RGB
ccm_black_rgb = [0, 0, 0]
ccm_dgray_rgb = [85, 85, 85]
ccm_gray_rgb = [210, 210, 210]
ccm_dblue_rgb = [25, 50, 120]
ccm_blue_rgb = [20, 80, 200]
ccm_lblue_rgb = [170, 200, 230]
ccm_red_rgb = [190, 0, 0]
ccm_orange_rgb = [255, 100, 0]

colors = [ccm_black_rgb, ccm_dgray_rgb, ccm_gray_rgb, ccm_dblue_rgb, ccm_blue_rgb, ccm_lblue_rgb, ccm_red_rgb,
          ccm_orange_rgb]
color = ['black', 'dgray', 'gray', 'dblue', 'blue', 'lblue', 'red', 'orange']

# Change decimal to binary
i = 0
for i in range(0, 8):
    color[i] = [colors[i] / 255 for colors[i] in colors[i]]

# Notification sounds
chime.theme('material')

### DATA PREPARATION

#### Load

In [ ]:
# DATA PREPARATION
# Load file
directory = "C:/Users/André/Documents/00_ITA/00_Mestrado/20_Data_Preparation/"
file_name = "top_force.csv"

file = pd.read_csv(directory + file_name)
print("\n# File head\n", file.head())

# Dropping unnecessary columns
main_df = file.copy()
main_df.drop(['Exp', 'Tool', 'Block', 'SBlock',
           'Position', 'Condition', 'TCond', 'Run',
           'Length', 'Di', 'Df', 'CTime', 'RAngle'],
           axis=1, inplace=True)
print("\n\n\n# File head after dropping unnecessary columns\n", main_df.head())

### Feature selection & scaling

In [6]:
# Define features and labels
features = ['ap', 'vc', 'f', 'Fx', 'Fy', 'Fz', 'F']
labels = ['Ra']
x_main = main_df.copy()[features]
y = main_df.copy()[labels]

# Scale x
sc_location = "C:/Users/André/Documents/00_ITA/00_Mestrado/70_pyproject/pyproject/"
sc_name = "standardize_all_data_wo_TCond.pkl"
sc = pickle.load(open(sc_location + sc_name, 'rb'))

x_array = sc.transform(x_main)
x = pd.DataFrame(data = x_array,
    columns = x_main.columns, index = x_main.index)

### MODEL BUILD

In [ ]:
# MODEL BUILD
# Clean session
keras.backend.clear_session()

# Define standard layers
Regularized_Dense = partial(keras.layers.Dense, activation = "relu")


# Funtion to create model
def create_model():
    '''This function creates a sequential model'''
    model = keras.Sequential()
    model.add(keras.Input(shape = x.shape[1:]))
    model.add(Regularized_Dense(28))
    # model.add(layers.Dropout(0.1))
    model.add(Regularized_Dense(56))
    # model.add(layers.Dropout(0.1))
    model.add(Regularized_Dense(14))
    # model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1))
    
    optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.95)
    
    model.compile(
        loss = "mean_squared_error",
        optimizer = optimizer,
        metrics = ["mean_absolute_percentage_error", "mean_absolute_error"])
    
    return model

checkpoint_path = "ANN1.H.L_full.hdf5"
cp = ModelCheckpoint(checkpoint_path,
                             monitor='mean_absolute_percentage_error',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=True,
                             mode='min')

es = EarlyStopping(monitor = "loss",
    min_delta = 0.0001,
    patience = 500,
    verbose = 1,
    mode = "min")

callbacks = [cp, es]

model = create_model()
model.summary()

### LEARN AND EVALUATE

In [ ]:
# LEARN
epochs = 10000
history = model.fit(x, y, epochs = epochs,
                    batch_size = 20, callbacks = callbacks,
                    verbose = 1)

# EVALUATE
loss = model.evaluate(x, y)
print('\nModel evaluation: \n', loss)
%chime

### MODEL SAVE

In [ ]:
model = create_model()
model.load_weights(checkpoint_path)
loss = model.evaluate(x, y, verbose=2)
# model.save("ANN1.H.L_full")